In [2]:
import pandas as pd

     ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
     ---------------------------------------- 0.1/10.7 MB 1.9 MB/s eta 0:00:06
      --------------------------------------- 0.2/10.7 MB 2.1 MB/s eta 0:00:06
      --------------------------------------- 0.2/10.7 MB 2.1 MB/s eta 0:00:06
      --------------------------------------- 0.2/10.7 MB 1.4 MB/s eta 0:00:08
     - -------------------------------------- 0.3/10.7 MB 1.3 MB/s eta 0:00:08
     - -------------------------------------- 0.4/10.7 MB 1.3 MB/s eta 0:00:08
     - -------------------------------------- 0.4/10.7 MB 1.2 MB/s eta 0:00:09
     - -------------------------------------- 0.5/10.7 MB 1.4 MB/s eta 0:00:08
     -- ------------------------------------- 0.6/10.7 MB 1.5 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/10.7 MB 1.6 MB/s eta 0:00:07
     --- ------------------------------------ 0.8/10.7 MB 1.7 MB/s eta 0:00:06
     --- ------------------------------------ 0.9/10.7 MB 1

In [1]:
data_file_path = 'train.txt'
data = pd.read_csv(data_file_path, sep='\t', header=None, names=['label', 'text'])
data['text'] = data['text'].astype(str)
print(data.head())

               label                                               text
0   TRUTHFULPOSITIVE  The sheraton was a wonderful hotel! When me an...
1   TRUTHFULPOSITIVE  We stayed at the Omni between Christmas and Ne...
2  DECEPTIVENEGATIVE  I was REALLY looking forward to a nice relaxin...
3   TRUTHFULNEGATIVE  First let me say, I try not to be too critical...
4  DECEPTIVENEGATIVE  The Ambassador East Hotel is a terrible place ...
